In [108]:
import pandas as pd
M = pd.read_pickle('../data/given/M.pkl')
N = None #pd.read_pickle('../data/given/N.pkl')
#test = pd.read_pickle('../data/given/test.pkl')

In [2]:
from common import *

In [3]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)

In [4]:
def setup_P(*,P):
    ##P.rename(columns={'returnsOpenNextMktres10':'y'}, inplace=True)
    P['y'] = P.returnsOpenNextMktres10
    #globalStd = (P.universe*P.y).std()
    #grp = P.groupby(F.assetCodeId)[['y','universe']]
    #assetStds = grp.apply(lambda x: np.sqrt((x.universe*x.y**2).sum()/(x.universe).sum()) if x.universe.sum()>25 else globalStd)
    #P['assetCodeId'] = P.assetCode.map(assetCodeIdAssign)
    #P.y /= P.assetCodeId.map(assetStds)
    P['target'] = P.y>0
    P['upDown'] = (P.target*2-1)
    P['upDown1'] = P.upDown*P.universe.astype(int)
    P['absVal'] = np.abs(P.y)
    P['absVal1'] = P.absVal*P.universe
    P['weight'] = P.absVal#.qtl()
    P['weight1'] = P.weight*P.universe

In [59]:
class IdAssign:
    class NO_MISSING(metaclass=staticclass): # special filler value because `None` might actually be meaningfully used
        pass
    def __init__(self, init, missing, name):
        self.name = name
        self.map = {}
        if missing is not __class__.NO_MISSING:
            self.map[missing] = -1
        i = 0
        for x in init:
            if x not in self.map and x != missing:
                self.map[x] = i
                i += 1
        self.cache = None
    def __call__(self, key):
        if key not in self.map:
            self.map[key] = len(self.map)
        return self.map[key]
    def __len__(self):
        return self.map.__len__()
    @property
    def series(self):
        if self.cache==len(self):
            return self._series
        # Python 3.?+ guarantees that dict keys and values are itered in same order, and that order is insertion order
        self._series = pd.Series(list(self.map.keys()), index=list(self.map.values()), name=self.name)
        self._series.index.name = self.name+'Id'
        self.cache = len(self)
        return self._series

assetCodeSeries = pd.Series(M.assetCode.unique())
assetCodeIdAssign = IdAssign(assetCodeSeries, missing='', name='assetCode')
del assetCodeSeries
assetNameSeries = pd.Series(M.assetName.unique())
assetNameIdAssign = IdAssign(assetNameSeries, missing='Unknown', name='assetName')
del assetNameSeries

In [60]:
stringify_columns = lambda f: '["'+'","'.join(c for c in f)+'"]'

returns_columns = {
    'returnsClosePrevRaw1':'cc', 'returnsOpenPrevRaw1':'oo',
    'returnsClosePrevMktres1':'acc', 'returnsOpenPrevMktres1':'aoo',
    'returnsClosePrevRaw10':'ccTEN','returnsOpenPrevRaw10':'ooTEN',
    'returnsClosePrevMktres10':'accTEN','returnsOpenPrevMktres10':'aooTEN'
}
columns_for_U = set(returns_columns.values()) | set(['open', 'close', 'volume'])
excluded_columns = [
    'time','assetCode','assetName','universe','returnsOpenNextMktres10','quarter','y'
]
exclusion_filter = lambda c: c not in excluded_columns
object_columns = ['assetCode', 'assetName']
enumeration_columns = ['assetCodeId', 'assetNameId']

In [61]:
train_start_time = pd.Timestamp('2009-01-01',tz='UTC')
lookback = 60
shortterm = 21
longterm = 250 + lookback*2 #idk it's hard to reason about how much I _really_ need, so this should be pretty safe

In [62]:
M['assetCodeId'] = M.assetCode.map(assetCodeIdAssign).astype(int)
inUniCount = M[M.time>=train_start_time].groupby('assetCodeId').universe.sum()
stocksInUni = (inUniCount != 0).pipe(lambda x: x.index[x])
assert stocksInUni.is_monotonic
xStocksInUni = set(stocksInUni) #EDITCELL
inUniCountSeries = inUniCount.astype(float) #EDITCELL
stocksAlways = (inUniCount >= 1981).pipe(lambda x: x.index[x])

In [109]:
def set_basic_features(*,M):
    M['assetCodeId'] = M.assetCode.map(assetCodeIdAssign)#.astype(int) # much more efficient to process everything as float
    #del M['assetCode']
    M['assetNameId'] = M.assetName.map(assetNameIdAssign)#.astype(int)
    #del M['assetName']
    for orig_col, new_col in returns_columns.items():
        M[new_col] = np.log1p(M[orig_col])
        #del M[orig_col]
    # time features
    M['dayOfYear'] = M.time.dt.dayofyear.astype(float)
    M['dayOfWeek'] = M.time.dt.dayofweek.astype(float)

In [110]:
set_basic_features(M=M)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in log1p
  import sys


## feature sets definitions

In [5]:
fThefault = ["volume","close","open","assetCodeId","assetNameId","cc","oo","cc0","oo0","cc_10","oo_10","cc0_10","oo0_10","dayOfYear","dayOfWeek","af","it","ooSince21","ooMaxSince21","ooDrawdownSince21","ooMinSince21","ooDrawupSince21","ooMaxDrawup5Since21","ooMaxDrawup10Since21","ooMaxDrawup(10-5)Since21","ooMaxDrawup(20-10)Since21","ooSince62","ooMaxSince62","ooDrawdownSince62","ooMinSince62","ooDrawupSince62","ooMaxDrawup5Since62","ooMaxDrawup10Since62","ooMaxDrawup(10-5)Since62","ooMaxDrawup(20-10)Since62","ooSince125","ooMaxSince125","ooDrawdownSince125","ooMinSince125","ooDrawupSince125","ooMaxDrawup5Since125","ooMaxDrawup10Since125","ooMaxDrawup(10-5)Since125","ooMaxDrawup(20-10)Since125","ooSince250","ooMaxSince250","ooDrawdownSince250","ooMinSince250","ooDrawupSince250","ooMaxDrawup5Since250","ooMaxDrawup10Since250","ooMaxDrawup(10-5)Since250","ooMaxDrawup(20-10)Since250","oo0Since21","oo0MaxSince21","oo0DrawdownSince21","oo0MinSince21","oo0DrawupSince21","oo0MaxDrawup5Since21","oo0MaxDrawup10Since21","oo0MaxDrawup(10-5)Since21","oo0MaxDrawup(20-10)Since21","oo0Since62","oo0MaxSince62","oo0DrawdownSince62","oo0MinSince62","oo0DrawupSince62","oo0MaxDrawup5Since62","oo0MaxDrawup10Since62","oo0MaxDrawup(10-5)Since62","oo0MaxDrawup(20-10)Since62","oo0Since125","oo0MaxSince125","oo0DrawdownSince125","oo0MinSince125","oo0DrawupSince125","oo0MaxDrawup5Since125","oo0MaxDrawup10Since125","oo0MaxDrawup(10-5)Since125","oo0MaxDrawup(20-10)Since125","oo0Since250","oo0MaxSince250","oo0DrawdownSince250","oo0MinSince250","oo0DrawupSince250","oo0MaxDrawup5Since250","oo0MaxDrawup10Since250","oo0MaxDrawup(10-5)Since250","oo0MaxDrawup(20-10)Since250","oo_{(0..4)^1}","oo_{(0..9)^1}","oo_{(0..14)^1}","oo_{(0..19)^1}","oo_{(5..9)^1}","oo_{(5..14)^1}","oo_{(5..19)^1}","oo_{(10..14)^1}","oo_{(10..19)^1}","oo_{(15..19)^1}","oo0_{(0..4)^1}","oo0_{(0..9)^1}","oo0_{(0..14)^1}","oo0_{(0..19)^1}","oo0_{(5..9)^1}","oo0_{(5..14)^1}","oo0_{(5..19)^1}","oo0_{(10..14)^1}","oo0_{(10..19)^1}","oo0_{(15..19)^1}","(oo-oo0)_{(0..4)^1}","(oo-oo0)_{(0..9)^1}","(oo-oo0)_{(0..14)^1}","(oo-oo0)_{(0..19)^1}","(oo-oo0)_{(5..9)^1}","(oo-oo0)_{(5..14)^1}","(oo-oo0)_{(5..19)^1}","(oo-oo0)_{(10..14)^1}","(oo-oo0)_{(10..19)^1}","(oo-oo0)_{(15..19)^1}","(oo-oo0){0/1}","(oo-oo0){0/2}","(oo-oo0){1/2}","(oo-oo0){0}","(oo-oo0){1}","(oo-oo0){2}","(cc-cc0){0/1}","(cc-cc0){0/2}","(cc-cc0){1/2}","(cc-cc0){0}","(cc-cc0){1}","(cc-cc0){2}","af{1}","it{1}","(it,af,it{1},oo0,cc0){0/1}","(it{1},af{1},it{2},oo0{1},cc0{1}){0/1}","(it,af,it{1},oo0,cc0){0/2}","(it{1},af{1},it{2},oo0{1},cc0{1}){0/2}","(it,af,it{1},oo0,cc0){0/3}","(it{1},af{1},it{2},oo0{1},cc0{1}){0/3}","(it,af,it{1},oo0,cc0){0/4}","(it{1},af{1},it{2},oo0{1},cc0{1}){0/4}","(it,af,it{1},oo0,cc0){1/2}","(it{1},af{1},it{2},oo0{1},cc0{1}){1/2}","(it,af,it{1},oo0,cc0){1/3}","(it{1},af{1},it{2},oo0{1},cc0{1}){1/3}","(it,af,it{1},oo0,cc0){1/4}","(it{1},af{1},it{2},oo0{1},cc0{1}){1/4}","(it,af,it{1},oo0,cc0){2/3}","(it{1},af{1},it{2},oo0{1},cc0{1}){2/3}","(it,af,it{1},oo0,cc0){2/4}","(it{1},af{1},it{2},oo0{1},cc0{1}){2/4}","(it,af,it{1},oo0,cc0){3/4}","(it{1},af{1},it{2},oo0{1},cc0{1}){3/4}","volumeRatioMeanBack2","volumeRatioMeanBack2{1}","volumeRatioMean5","volumeRatioMean10","volumeRatioMean10{5}","volumeRatioMean5ByMean20","oo0_20","oo0_60","oo0_120","oo0_250","ooMaxDrawdown5Since21","ooMaxDrawdown10Since21","ooMaxDrawdown(10-5)Since21","ooMaxDrawdown(20-10)Since21","ooPrev5MaxSince21","ooPrev10MaxSince21","ooPrev5MinSince21","ooPrev10MinSince21","ooMaxDrawdown5Since62","ooMaxDrawdown10Since62","ooMaxDrawdown(10-5)Since62","ooMaxDrawdown(20-10)Since62","ooPrev5MaxSince62","ooPrev10MaxSince62","ooPrev5MinSince62","ooPrev10MinSince62","ooMaxDrawdown5Since125","ooMaxDrawdown10Since125","ooMaxDrawdown(10-5)Since125","ooMaxDrawdown(20-10)Since125","ooPrev5MaxSince125","ooPrev10MaxSince125","ooPrev5MinSince125","ooPrev10MinSince125","ooMaxDrawdown5Since250","ooMaxDrawdown10Since250","ooMaxDrawdown(10-5)Since250","ooMaxDrawdown(20-10)Since250","ooPrev5MaxSince250","ooPrev10MaxSince250","ooPrev5MinSince250","ooPrev10MinSince250","oo0MaxDrawdown5Since21","oo0MaxDrawdown10Since21","oo0MaxDrawdown(10-5)Since21","oo0MaxDrawdown(20-10)Since21","oo0Prev5MaxSince21","oo0Prev10MaxSince21","oo0Prev5MinSince21","oo0Prev10MinSince21","oo0MaxDrawdown5Since62","oo0MaxDrawdown10Since62","oo0MaxDrawdown(10-5)Since62","oo0MaxDrawdown(20-10)Since62","oo0Prev5MaxSince62","oo0Prev10MaxSince62","oo0Prev5MinSince62","oo0Prev10MinSince62","oo0MaxDrawdown5Since125","oo0MaxDrawdown10Since125","oo0MaxDrawdown(10-5)Since125","oo0MaxDrawdown(20-10)Since125","oo0Prev5MaxSince125","oo0Prev10MaxSince125","oo0Prev5MinSince125","oo0Prev10MinSince125","oo0MaxDrawdown5Since250","oo0MaxDrawdown10Since250","oo0MaxDrawdown(10-5)Since250","oo0MaxDrawdown(20-10)Since250","oo0Prev5MaxSince250","oo0Prev10MaxSince250","oo0Prev5MinSince250","oo0Prev10MinSince250"]
print('fThefault')

fThefault


In [6]:
fNewfault = ["volume","close","open","assetCodeId","assetNameId","cc","oo","cc0","oo0","cc_10","oo_10","cc0_10","oo0_10","dayOfYear","dayOfWeek","ooSince21","ooMaxSince21","ooDrawdownSince21","ooMaxDrawdown5Since21","ooMaxDrawdown10Since21","ooPrev10MaxSince21","ooMinSince21","ooPrev10MinSince21","ooSince62","ooMaxSince62","ooDrawdownSince62","ooMaxDrawdown5Since62","ooMaxDrawdown10Since62","ooPrev10MaxSince62","ooMinSince62","ooPrev10MinSince62","ooSince125","ooMaxSince125","ooDrawdownSince125","ooMaxDrawdown5Since125","ooMaxDrawdown10Since125","ooPrev10MaxSince125","ooMinSince125","ooPrev10MinSince125","ooSince250","ooMaxSince250","ooDrawdownSince250","ooMaxDrawdown5Since250","ooMaxDrawdown10Since250","ooPrev10MaxSince250","ooMinSince250","ooPrev10MinSince250","oo0Since21","oo0MaxSince21","oo0DrawdownSince21","oo0MaxDrawdown5Since21","oo0MaxDrawdown10Since21","oo0Prev10MaxSince21","oo0MinSince21","oo0Prev10MinSince21","oo0Since62","oo0MaxSince62","oo0DrawdownSince62","oo0MaxDrawdown5Since62","oo0MaxDrawdown10Since62","oo0Prev10MaxSince62","oo0MinSince62","oo0Prev10MinSince62","oo0Since125","oo0MaxSince125","oo0DrawdownSince125","oo0MaxDrawdown5Since125","oo0MaxDrawdown10Since125","oo0Prev10MaxSince125","oo0MinSince125","oo0Prev10MinSince125","oo0Since250","oo0MaxSince250","oo0DrawdownSince250","oo0MaxDrawdown5Since250","oo0MaxDrawdown10Since250","oo0Prev10MaxSince250","oo0MinSince250","oo0Prev10MinSince250","oo_{(0..4)^1}","oo_{(0..9)^1}","oo_{(0..14)^1}","oo_{(0..19)^1}","oo_{(5..9)^1}","oo_{(5..14)^1}","oo_{(5..19)^1}","oo_{(10..14)^1}","oo_{(10..19)^1}","oo_{(15..19)^1}","oo0_{(0..4)^1}","oo0_{(0..9)^1}","oo0_{(0..14)^1}","oo0_{(0..19)^1}","oo0_{(5..9)^1}","oo0_{(5..14)^1}","oo0_{(5..19)^1}","oo0_{(10..14)^1}","oo0_{(10..19)^1}","oo0_{(15..19)^1}","(it,af,it{1},oo0,cc0){3/4}"]
fNewnewfault = fNewfault[:-1] + ['(oo0-cc0)']
print('fNewfault')

fNewfault


In [8]:
fTransNewfault = ["volume","close","open","assetCodeId","assetNameId","cc","oo","acc","aoo","ccTEN","ooTEN","accTEN","aooTEN","dayOfYear","dayOfWeek","oo1Since21","oo1MaxSince21","oo1Drawdown1Since21","oo1Drawdown5Since21","oo1Drawdown10Since21","oo1MaxSince21{10}","oo1MinSince21","oo1MinSince21{10}","oo1Since62","oo1MaxSince62","oo1Drawdown1Since62","oo1Drawdown5Since62","oo1Drawdown10Since62","oo1MaxSince62{10}","oo1MinSince62","oo1MinSince62{10}","oo1Since125","oo1MaxSince125","oo1Drawdown1Since125","oo1Drawdown5Since125","oo1Drawdown10Since125","oo1MaxSince125{10}","oo1MinSince125","oo1MinSince125{10}","oo1Since250","oo1MaxSince250","oo1Drawdown1Since250","oo1Drawdown5Since250","oo1Drawdown10Since250","oo1MaxSince250{10}","oo1MinSince250","oo1MinSince250{10}","aoo1Since21","aoo1MaxSince21","aoo1Drawdown1Since21","aoo1Drawdown5Since21","aoo1Drawdown10Since21","aoo1MaxSince21{10}","aoo1MinSince21","aoo1MinSince21{10}","aoo1Since62","aoo1MaxSince62","aoo1Drawdown1Since62","aoo1Drawdown5Since62","aoo1Drawdown10Since62","aoo1MaxSince62{10}","aoo1MinSince62","aoo1MinSince62{10}","aoo1Since125","aoo1MaxSince125","aoo1Drawdown1Since125","aoo1Drawdown5Since125","aoo1Drawdown10Since125","aoo1MaxSince125{10}","aoo1MinSince125","aoo1MinSince125{10}","aoo1Since250","aoo1MaxSince250","aoo1Drawdown1Since250","aoo1Drawdown5Since250","aoo1Drawdown10Since250","aoo1MaxSince250{10}","aoo1MinSince250","aoo1MinSince250{10}","oo5","oo10","oo15","oo20","oo(10-5)","oo(15-5)","oo(20-5)","oo(15-10)","oo(20-10)","oo(20-15)","aoo5","aoo10","aoo15","aoo20","aoo(10-5)","aoo(15-5)","aoo(20-5)","aoo(15-10)","aoo(20-10)","aoo(20-15)","(it,af,it{1},aoo,acc){3/4}"]
fTransNewnewfault = fTransNewfault[:-1] + ['(aoo-acc)']
print('fTransNewfault')

fTransNewfault


In [9]:
f2firstkaggle = ["assetCodeId","volume","close","open","assetNameId","cc","oo","cc0","oo0","cc_10","oo_10","cc0_10","oo0_10","dayOfYear","dayOfWeek","af","it","ooSince21","ooMaxSince21","ooDrawdownSince21","ooMinSince21","ooDrawupSince21","ooMaxDrawup10Since21","ooMaxDrawup(20-10)Since21","ooSince125","ooMaxSince125","ooDrawdownSince125","ooMinSince125","ooDrawupSince125","ooMaxDrawup10Since125","ooMaxDrawup(20-10)Since125","oo_{(0..4)^1}","oo_{(0..9)^1}","oo_{(0..14)^1}","oo_{(0..19)^1}","oo_{(5..9)^1}","oo_{(5..14)^1}","oo_{(5..19)^1}","oo_{(10..14)^1}","oo_{(10..19)^1}","oo_{(15..19)^1}","oo0_{(0..4)^1}","oo0_{(0..9)^1}","oo0_{(0..14)^1}","oo0_{(0..19)^1}","oo0_{(5..9)^1}","oo0_{(5..14)^1}","oo0_{(5..19)^1}","oo0_{(10..14)^1}","oo0_{(10..19)^1}","oo0_{(15..19)^1}","(oo-oo0)_{(0..4)^1}","(oo-oo0)_{(0..9)^1}","(oo-oo0)_{(0..14)^1}","(oo-oo0)_{(0..19)^1}","(oo-oo0)_{(5..9)^1}","(oo-oo0)_{(5..14)^1}","(oo-oo0)_{(5..19)^1}","(oo-oo0)_{(10..14)^1}","(oo-oo0)_{(10..19)^1}","(oo-oo0)_{(15..19)^1}","(it,af,it{1},oo0,cc0){0/1}","(it,af,it{1},oo0,cc0){0/2}","(it,af,it{1},oo0,cc0){0/3}","(it,af,it{1},oo0,cc0){0/4}","(it,af,it{1},oo0,cc0){1/2}","(it,af,it{1},oo0,cc0){1/3}","(it,af,it{1},oo0,cc0){1/4}","(it,af,it{1},oo0,cc0){2/3}","(it,af,it{1},oo0,cc0){2/4}","(it,af,it{1},oo0,cc0){3/4}","volumeRatioMeanBack2","volumeRatioMeanBack2{1}","volumeRatioMean5","volumeRatioMean10","oo0_20","oo0_120","oo_20","oo_120","(oo-oo0)_10","(oo-oo0)_20","(oo-oo0)_120","ooMaxDrawdown10Since21","ooMaxDrawdown(20-10)Since21","ooPrev10MaxSince21","ooPrev10MinSince21","ooMaxDrawdown10Since125","ooMaxDrawdown(20-10)Since125","ooPrev10MaxSince125","ooPrev10MinSince125"]#,"assetCodeIdWTFlogabs","assetCodeIdWTFsin2","assetCodeIdWTFFlogabs","assetCodeIdWTFFsin2"]
print('f2firstkaggle')

f2firstkaggle


In [10]:
fTime = 'dayOfYear dayOfWeek'.split()
fAsset = 'assetCodeId assetNameId'.split()
fPriceAbsolute = 'close open'.split()
print('fTime fAsset fPriceAbsolute')

fTime fAsset fPriceAbsolute


In [11]:
def select_relative(f):
    return f not in set(fTime+fAsset+fPriceAbsolute)

fThefaultRelative = list(filter(select_relative, fThefault))

fHack1 = ['assetCodeIdWTFlogabs', 'assetCodeIdWTFsin2']
fHack2 = ['assetCodeIdWTFFlogabs', 'assetCodeIdWTFFsin2']
fNewfaultRelativeHack0 = list(filter(select_relative, fNewfault)) + fHack2

print('select_relative combinations... |')

select_relative combinations... |


##### choose our feature selection:

In [75]:
fSelection0 = list(filter(select_relative,fNewnewfault)) + ['assetCodeId','assetCodeIdWTFFlogabs', 'assetCodeIdWTFFsin2']
fSelection1 = list(filter(select_relative,fTransNewnewfault))+['assetCodeId','assetCodeIdWTFFlogabs', 'assetCodeIdWTFFsin2']

## load the data frames to compare

In [15]:
F,P = pd.read_pickle('/big/data/saves/train_7isfinalmaybe.32.pkl')

#### need to make sure feature generation is bug-free

In [16]:
vct = F.nunique()

In [53]:
# reference code to examine column
c = '(oo-aoo)10'
pd.concat([F[c].groupby(P.time).first()], axis=1)
F.m_oo

732812     0.0
732814     0.0
732815     0.0
732816     0.0
732817     0.0
732818     0.0
732819     0.0
732820     0.0
732821     0.0
732822     0.0
732823     0.0
732824     0.0
732825     0.0
732826     0.0
732827     0.0
732828     0.0
732829     0.0
732830     0.0
732831     0.0
732832     0.0
732833     0.0
732834     0.0
732835     0.0
732836     0.0
732837     0.0
732838     0.0
732839     0.0
732840     0.0
732841     0.0
732842     0.0
732843     0.0
732844     0.0
732845     0.0
732848     0.0
732849     0.0
732850     0.0
732851     0.0
732852     0.0
732854     0.0
732855     0.0
732856     0.0
732857     0.0
732858     0.0
732859     0.0
732860     0.0
732863     0.0
732866     0.0
732867     0.0
732868     0.0
732869     0.0
732870     0.0
732871     0.0
732872     0.0
732874     0.0
732875     0.0
732876     0.0
732877     0.0
732878     0.0
732881     0.0
732882     0.0
732883     0.0
732884     0.0
732885     0.0
732886     0.0
732888     0.0
732889     0.0
732890    

In [24]:
F[F.assetCodeId==3].oo1MinSince62

732815    -0.238038
734404    -0.115435
735994    -0.203804
737587    -0.047878
739182    -0.131608
740777    -0.044383
742373    -0.245605
743970    -0.350507
745564    -0.236402
747159    -0.214711
748755    -0.211197
750350    -0.215812
751947    -0.198645
753546    -0.201734
755147    -0.194304
756746    -0.126494
758346    -0.178856
759950    -0.182007
761554    -0.236786
763160    -0.310071
764762    -0.300524
766369    -0.286833
767980    -0.324632
769592    -0.315591
771204    -0.239858
772817    -0.220877
774437    -0.232409
776059    -0.176329
777679    -0.151452
779300    -0.124053
780924    -0.163594
782552    -0.107246
784185    -0.119598
785824    -0.120491
787464    -0.067717
789108    -0.030596
790754    -0.068070
792396    -0.174112
794038    -0.125163
795682    -0.174957
797332    -0.138722
798984    -0.124053
800637    -0.116916
802290    -0.171360
803942    -0.127380
805594    -0.200396
807247    -0.208802
808898    -0.207474
810548    -0.200499
812198    -0.155659


In [17]:
vct

assetCodeId                                  2267
volume                                    1628842
close                                       42773
open                                        36708
assetNameId                                  2128
cc                                         812834
oo                                         805310
acc                                       1956978
aoo                                       1956948
ccTEN                                     1283415
ooTEN                                     1269963
accTEN                                    1950730
aooTEN                                    1950356
dayOfYear                                     365
dayOfWeek                                       5
ooTEN1                                    1269963
oo1                                        805310
aoo1                                      1956948
accTEN1                                   1950730
ccTEN1                                    1283415


#### more detailed comparison

start load

In [21]:
F0, P0 = pd.read_pickle('/big/data/saves/train_4Newnewfault64xx.pkl')

end load

In [22]:
set(F0.columns)^set(fSelection)

{'assetCodeId',
 'assetCodeIdWTFlogabs',
 'assetCodeIdWTFsin2',
 'assetNameId',
 'close',
 'dayOfWeek',
 'dayOfYear',
 'open'}

In [45]:
mp = dict(zip(fSelection1,fSelection0))

In [76]:
gc.collect()
df = F[fSelection1].rename(columns=mp)
df0 = pd.DataFrame(index=df.index).join(F0[fSelection0], how='left')
p0 = pd.DataFrame(index=df.index).join(P0)
eq = np.abs(df0.fillna(np.inf)-df.fillna(np.inf)).fillna(0)<1e-8

In [92]:
(eq.sum(axis=0)/len(eq)).pipe(lambda x: x[x<.99])

volume                      0.980112
ooSince21                   0.020194
ooMaxSince21                0.019106
ooDrawdownSince21           0.148811
ooMaxDrawdown5Since21       0.010816
ooMaxDrawdown10Since21      0.001029
ooPrev10MaxSince21          0.018731
ooMinSince21                0.018377
ooPrev10MinSince21          0.017889
ooSince62                   0.029412
ooMaxSince62                0.028942
ooDrawdownSince62           0.098016
ooMaxDrawdown5Since62       0.006511
ooMaxDrawdown10Since62      0.000638
ooPrev10MaxSince62          0.029151
ooMinSince62                0.028783
ooPrev10MinSince62          0.028880
ooSince125                  0.040670
ooMaxSince125               0.040217
ooDrawdownSince125          0.077984
ooMaxDrawdown5Since125      0.005185
ooMaxDrawdown10Since125     0.000576
ooPrev10MaxSince125         0.040590
ooMinSince125               0.041033
ooPrev10MinSince125         0.041279
ooSince250                  0.061614
ooMaxSince250               0.061658
o

In [69]:
M[M.assetCodeId==3].pipe(lambda x: x[x.time>=train_start_time]).oo

732815    -0.004896
734404     0.084330
735994     0.030150
737587    -0.046500
739182    -0.012524
740777     0.029947
742373    -0.030058
743970    -0.023604
745564    -0.024059
747159    -0.068877
748755     0.045769
750350    -0.027673
751947    -0.030596
753546     0.101580
755147    -0.012004
756746     0.021954
758346     0.013633
759950     0.022498
761554     0.010149
763160    -0.005062
764762    -0.039869
766369     0.030867
767980     0.016531
769592    -0.005803
771204     0.045201
772817     0.029738
774437     0.011632
776059    -0.048296
777679    -0.005930
779300     0.033655
780924    -0.021519
782552    -0.019256
784185    -0.019420
785824    -0.041908
787464     0.023304
789108    -0.048654
790754     0.029475
792396     0.024731
794038    -0.045647
795682    -0.000114
797332     0.012536
798984     0.000000
800637     0.012824
802290    -0.021504
803942    -0.049668
805594     0.008648
807247     0.058638
808898     0.032826
810548     0.035945
812198     0.002178


In [122]:
THE = 3
guy = pd.concat([
    P.time[F.assetCodeId==THE],
    df[F.assetCodeId==THE][['oo','ooSince21']],
    #p0.time[F.assetCodeId==THE],
    df0[F.assetCodeId==THE][['ooSince21']],
    pd.DataFrame(index=df.index).join(M)[F.assetCodeId==THE][['assetCode','returnsOpenPrevRaw1']]
], axis=1)
#assert (guy.iloc[:,0]==guy.iloc[:,3]).all()
guy.index = np.arange(len(guy))
guy

,time,oo,ooSince21,ooSince21,assetCode,returnsOpenPrevRaw1
0,2009-01-02 22:00:00+00:00,-0.004896,-0.065028,-0.050358,AAPL.O,-0.004884
1,2009-01-05 22:00:00+00:00,0.084330,0.033972,0.041108,AAPL.O,0.087988
2,2009-01-06 22:00:00+00:00,0.030150,0.071258,0.016814,AAPL.O,0.030609
3,2009-01-07 22:00:00+00:00,-0.046500,-0.029686,0.014294,AAPL.O,-0.045436
4,2009-01-08 22:00:00+00:00,-0.012524,0.001770,-0.071245,AAPL.O,-0.012445
5,2009-01-09 22:00:00+00:00,0.029947,-0.041298,-0.049704,AAPL.O,0.030400
6,2009-01-12 22:00:00+00:00,-0.030058,-0.079762,-0.078434,AAPL.O,-0.029611
7,2009-01-13 22:00:00+00:00,-0.023604,-0.102038,-0.095063,AAPL.O,-0.023328
8,2009-01-14 22:00:00+00:00,-0.024059,-0.119122,-0.074282,AAPL.O,-0.023772
9,2009-01-15 22:00:00+00:00,-0.068877,-0.143159,-0.175883,AAPL.O,-0.066558


In [127]:
guy['ooRolling'] = guy.oo.rolling(21).sum()
guy['ooCumsum'] = guy.oo.cumsum()
guy

,time,oo,ooSince21,ooSince21,assetCode,returnsOpenPrevRaw1,ooCumsum,ooRolling
0,2009-01-02 22:00:00+00:00,-0.004896,-0.065028,-0.050358,AAPL.O,-0.004884,-0.004896,NaN
1,2009-01-05 22:00:00+00:00,0.084330,0.033972,0.041108,AAPL.O,0.087988,0.079434,NaN
2,2009-01-06 22:00:00+00:00,0.030150,0.071258,0.016814,AAPL.O,0.030609,0.109584,NaN
3,2009-01-07 22:00:00+00:00,-0.046500,-0.029686,0.014294,AAPL.O,-0.045436,0.063084,NaN
4,2009-01-08 22:00:00+00:00,-0.012524,0.001770,-0.071245,AAPL.O,-0.012445,0.050560,NaN
5,2009-01-09 22:00:00+00:00,0.029947,-0.041298,-0.049704,AAPL.O,0.030400,0.080508,NaN
6,2009-01-12 22:00:00+00:00,-0.030058,-0.079762,-0.078434,AAPL.O,-0.029611,0.050450,NaN
7,2009-01-13 22:00:00+00:00,-0.023604,-0.102038,-0.095063,AAPL.O,-0.023328,0.026846,NaN
8,2009-01-14 22:00:00+00:00,-0.024059,-0.119122,-0.074282,AAPL.O,-0.023772,0.002787,NaN
9,2009-01-15 22:00:00+00:00,-0.068877,-0.143159,-0.175883,AAPL.O,-0.066558,-0.066090,NaN


old:

In [57]:
dude = M[M.assetCode=='AAPL.O'][M.time<'2009-01-03'].tail(5)
set_basic_features(M=dude)
dude

,time,assetCode,assetName,volume,close,open,returnsOpenNextMktres10,universe,assetCodeId,assetNameId,cc,oo,cc0,oo0,cc_10,oo_10,cc0_10,oo0_10,dayOfYear,dayOfWeek
726443,2008-12-26 22:00:00+00:00,AAPL.O,Apple Inc,11043071.0,85.81,86.55,0.023792,1.0,3,3,0.009014,0.005213,0.005038,0.001207,-0.101741,-0.115534,-0.100484,-0.078656,361.0,4.0
728035,2008-12-29 22:00:00+00:00,AAPL.O,Apple Inc,24567225.0,86.61,86.62,-0.008313,1.0,3,3,0.009280,0.000808,0.011972,-0.001201,-0.126304,-0.069885,-0.109256,-0.070672,364.0,0.0
729627,2008-12-30 22:00:00+00:00,AAPL.O,Apple Inc,34579620.0,86.29,87.42,0.008330,1.0,3,3,-0.003702,0.009193,-0.020864,0.010283,-0.093528,-0.093416,-0.130348,-0.078830,365.0,1.0
731221,2008-12-31 22:00:00+00:00,AAPL.O,Apple Inc,21715763.0,85.35,86.00,0.052227,1.0,3,3,-0.010953,-0.016377,-0.023658,-0.028226,-0.111633,-0.088947,-0.093581,-0.116575,366.0,2.0
732815,2009-01-02 22:00:00+00:00,AAPL.O,Apple Inc,26964210.0,90.75,85.58,-0.026166,1.0,3,3,0.061348,-0.004896,0.032733,-0.015290,0.017676,-0.061408,-0.029549,-0.053305,2.0,4.0


In [58]:
dude.oo.sum()

-0.0060578004450126009

In [63]:
c = '(oo0-cc0)'
guy = pd.concat({'time':P.time,'assetCodeId':F.assetCodeId,'df':df[c],'df0': df0[c],'cc0':df.cc0,'oo0':df.oo0}, axis=1)
guy[(guy.assetCodeId!=-1) & (df[c]!=df0[c])].pipe(lambda x: pd.concat([x, x.oo0-x.cc0], axis=1))

,assetCodeId,cc0,df,df0,oo0,time,0
732812,0,0.028696,1.256251e-02,-0.064784,4.125897e-02,2009-01-02 22:00:00+00:00,1.256251e-02
732813,1,-0.018934,-3.017972e-02,0.042388,-4.911375e-02,2009-01-02 22:00:00+00:00,-3.017972e-02
732814,2,-0.010750,1.986397e-02,0.021261,9.114077e-03,2009-01-02 22:00:00+00:00,1.986397e-02
732815,3,0.032733,-4.802233e-02,-0.004568,-1.528981e-02,2009-01-02 22:00:00+00:00,-4.802233e-02
732816,1564,-0.055761,1.097132e-01,0.002326,5.395219e-02,2009-01-02 22:00:00+00:00,1.097132e-01
732817,4,-0.017254,3.152896e-02,-0.011766,1.427474e-02,2009-01-02 22:00:00+00:00,3.152896e-02
732818,5,-0.017041,3.983991e-02,0.004516,2.279893e-02,2009-01-02 22:00:00+00:00,3.983991e-02
732819,7,-0.045668,4.680588e-02,-0.029059,1.138005e-03,2009-01-02 22:00:00+00:00,4.680588e-02
732820,2099,-0.042891,9.170951e-02,-0.053478,4.881807e-02,2009-01-02 22:00:00+00:00,9.170951e-02
732821,8,-0.008815,7.970818e-03,0.010375,-8.440820e-04,2009-01-02 22:00:00+00:00,7.970818e-03


In [58]:
for c in combos.omo(5):
    name = '(it,af,it{{1}},oo0,cc0){{{}}}'.format(combos.name.index(c))
    if name == '(it,af,it{1},oo0,cc0){3/4}':
        print(c)
        print(c@np.asarray([[0]*9,[1]*9,[2]*9,[4]*9,[8.]*9]))

[ 0  0  0  1 -1]
[-4. -4. -4. -4. -4. -4. -4. -4. -4.]
